In [92]:
import pandas as pd
from thefuzz import fuzz

In [34]:
divs = pd.read_csv('section_boundaries.csv', index_col=0)[508:]

In [35]:
divs

,datecode,type,start_phrase,end_phrase
508,20170920,FOMC2,"Thank you, Madam Chair. I will be referring to...","Thank you, Madam Chair. As you noted, this vot..."
509,20170920,FOMC1,All in favor? [Chorus of ayes] Any opposed? [N...,"Well, my thanks to everyone for a rich discuss..."
510,20171101,FOMC1,Thank you. All in favor? [Chorus of ayes] Any ...,Thank you. My thanks to everyone for a thought...
511,20171101,FOMC2,"Good morning, everybody. Let’s get started. An...","Thank you, Madam Chair. As you indicated, this..."
512,20171213,FOMC1,Any questions for David? [No response] Okay. S...,"Thank you. Well, my thanks to everyone for a t..."
513,20171213,FOMC2,Any questions for David? [No response] Okay. S...,"Thank you, Madam Chair. The vote will be on th..."


In [52]:
new = divs.pivot(index='datecode', columns='type', values=['start_phrase', 'end_phrase']).reset_index()

In [143]:
res_dfs = []
for group in pd.read_csv('../working-csvs/raw_transcripts.csv', index_col=0).groupby('date'):
    datecode = int(group[0].replace('-', '')) 
    if datecode not in divs['datecode'].unique():
        continue
    FOMC1_bounds = [new.loc[new.datecode == datecode].start_phrase.FOMC1.iloc[0], new.loc[new.datecode == datecode].start_phrase.FOMC2.iloc[0]]
    FOMC2_bounds = [new.loc[new.datecode == datecode].end_phrase.FOMC1.iloc[0], new.loc[new.datecode == datecode].end_phrase.FOMC2.iloc[0]]
    sections = [FOMC1_bounds, FOMC2_bounds]
    transc = group[1]
    for i, section in enumerate(sections):
        transc[f'sec{i + 1}startsim'] = transc['content'].apply(lambda x : fuzz.ratio(x, section[0]))
        transc[f'sec{i + 1}endsim'] = transc['content'].apply(lambda x : fuzz.ratio(x, section[1]))
    maxes = transc.idxmax(numeric_only=True)
    FOMC1 = transc.loc[maxes.sec1startsim:maxes.sec1endsim].copy()
    FOMC1['section'] = 1
    FOMC2 = transc.loc[maxes.sec2startsim:maxes.sec2endsim].copy()
    FOMC2['section'] = 2
    result = pd.concat([FOMC1, FOMC2])
    result = result.loc[:, ~result.columns.isin(['sec1startsim', 'sec1endsim', 'sec2startsim', 'sec2endsim'])]
    res_dfs.append(result)

In [147]:
pd.concat(res_dfs).to_csv('../working-csvs/raw_transcripts_sectioned.csv')